# Basic
Importing libraries and setting up the simulation material

In [67]:
import sys, os
import importlib.util
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import lambda2nu, nu2lambda, c

In [68]:
from IPython.core.display import HTML
from IPython.core.pylabtools import figsize
from IPython.display import display, Math

In [69]:
lumapiFile = r"C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py"
spec = importlib.util.spec_from_file_location("lumapi", lumapiFile)
lumapi = importlib.util.module_from_spec(spec)
spec.loader.exec_module(lumapi)
modeapi = lumapi.MODE(hide = False)

C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py:796: SyntaxWarning: invalid escape sequence '\s'
  message = re.sub('^(Error:)\s(prompt line)\s[0-9]+:', '', str(rvals[2])).strip()


In [70]:
um = 1e-6
nm = 1e-9

In [87]:
# Materials
material_Clad = "SiO2 (Glass) - Palik"
material_Si = "Si (Silicon) - Dispersive & Lossless"

# Add material Si
matname = material_Si
matReturn = modeapi.getmaterial(material_Si)
if (modeapi.findstring(matReturn, 'is not available') != -1):
    newmaterial = modeapi.addmaterial("Lorentz")
    modeapi.setmaterial(newmaterial, "name", matname)
    modeapi.setmaterial(matname, "Permittivity", 7.98737492)
    modeapi.setmaterial(matname, "Lorentz Linewidth", 1e8)
    modeapi.setmaterial(matname, "Lorentz Resonance", 3.93282466e+15)
    modeapi.setmaterial(matname, "Lorentz Permittivity", 3.68799143)
    modeapi.setmaterial(matname, "color", np.array([0.85, 0, 0, 1]))  # red

In [88]:
# Add material Clad
matname = material_Clad
matReturn = modeapi.getmaterial(material_Clad)
if (modeapi.findstring(matReturn, 'is not available') != -1):
    newmaterial = modeapi.addmaterial("Lorentz")
    modeapi.setmaterial(newmaterial, "name", matname)
    modeapi.setmaterial(matname, "Permittivity", 2.119881)
    modeapi.setmaterial(matname, "Lorentz Linewidth", 1e10)
    modeapi.setmaterial(matname, "Lorentz Resonance", 3.30923843e+15)
    modeapi.setmaterial(matname, "Lorentz Permittivity", 49.43721)
    modeapi.setmaterial(matname, "color", np.array([0.5, 0.5, 0.5, 1]))  # grey

In [117]:
modeapi.switchtolayout()

modeapi.deleteall()

modeapi.addstructuregroup()
dev_layer = 'taper_cross'
modeapi.set('name',dev_layer)
modeapi.set("x", 0)
modeapi.set("y", 0)
modeapi.set("z", 0)


# Measurements

In [118]:
#wavelenght
lambda_0 = 1550*nm

# Waveguide parameters
wvg_width = 500 * nm
wvg_height = 220 * nm
wvg_lenght = 9 * um

# Crossing parameters
cross_lenght = 6 * um
shlwetch_width = 800 * nm #the shallow etched final width when it gets to the crossing
curve_width = 2.5 * um
shlwetch_depth = 50 * nm
deepetch_depth = 220 * nm

In [120]:
# C Band simulation: 1520 - 1575 nm
# L Band simulation: 1555 - 1635 nm
# Bandwidth operation
cband = True
filename = "final_project_%dx%d_nm" % ( shlwetch_depth / nm ,deepetch_depth / nm)
filename_var = "final_projectVarFDTD_%dx%d_nm" % (shlwetch_depth / nm , deepetch_depth / nm )

if cband:
    wvlngth_start = 1.520 * um
    wvlngth_stop = 1.575 * um
else:
    wvlngth_start = 1.555 * um
    wvlngth_stop = 1.635 * um

numFreqPnts = 20

In [128]:
modeapi.select("Glass_palik")
modeapi.delete()

modeapi.addrect()
modeapi.set("name", "Glass_palik")
modeapi.set("x", 0)
modeapi.set("x span", 12 * um)
modeapi.set("y", 0)
modeapi.set("y span", 12 * um)
modeapi.set("z", -1*um)
modeapi.set("z span",2 * um)
modeapi.set("material", material_Clad)

# Adding waveguides

In [126]:

# Middle square
modeapi.select(f"{dev_layer}::box")
modeapi.delete()
modeapi.addrect()
modeapi.addtogroup(dev_layer)
modeapi.set("name", "box")
modeapi.set("x", 0)
modeapi.set("x span", shlwetch_width)
modeapi.set("y", 0)
modeapi.set("y span", shlwetch_width)
modeapi.set("z", (wvg_height) / 2)
modeapi.set("z span",(wvg_height))
modeapi.set("material", material_Si)


In [133]:
# Tapers
for i in range(4):
    
    #adding the shallow etch 
    modeapi.select(f"{dev_layer}::taper port {str(i + 1)}")
    modeapi.delete()
    modeapi.addobject("linear_taper")
    modeapi.addtogroup(dev_layer)
    modeapi.set("name", f"taper port {str(i + 1)}")
    modeapi.set("thickness", wvg_height)
    modeapi.set("angle_side", 90)
    modeapi.set("width_l", shlwetch_width)
    modeapi.set("width_r", wvg_width)
    modeapi.set("len", (cross_lenght-(800*nm))/2)
    modeapi.set("material", material_Si)
    modeapi.set("z", (wvg_height) / 2)
    
    # making the first 2 tapers be horizontal and the last 2 vertical
    if(i<2):
        x0 =  modeapi.getnamed(f"{dev_layer}::box", "x max")+ modeapi.getnamed(f"{dev_layer}::taper port {str(i + 1)}", "len")/2
        y0 = modeapi.getnamed(f"{dev_layer}::box", "y")    
        yspan = wvg_width
        xspan = 2 * um
        x1 = modeapi.getnamed(f"{dev_layer}::box", "x") + (cross_lenght+ 2 * um)/2 
        y1 = modeapi.getnamed(f"{dev_layer}::box", "y")
    else:
        y0 =  modeapi.getnamed(f"{dev_layer}::box", "y max")+ modeapi.getnamed(f"{dev_layer}::taper port {str(i + 1)}", "len")/2
        x0 = modeapi.getnamed(f"{dev_layer}::box", "x")
        modeapi.set("first axis", "z")
        modeapi.set("rotation 1", 90)
        xspan = wvg_width
        yspan = 2 * um
        x1 = modeapi.getnamed(f"{dev_layer}::box", "x")
        y1 = modeapi.getnamed(f"{dev_layer}::box", "y") + (cross_lenght+ 2 * um)/2

# fixing their start and end
    if((i + 1) % 2 == 0):
        modeapi.set("x", -x0)
        modeapi.set("y", y0)
        modeapi.set("second axis", "y")
        modeapi.set("rotation 2", 180)
        x1 = -x1
        #y1 = y1
    else:
        modeapi.set("x", x0)
        modeapi.set("y", -y0)
        modeapi.set("second axis", "x")
        modeapi.set("rotation 2", 180)
        y1 = -y1
        
    modeapi.select(f"{dev_layer}::waveguide {str(i + 1)}")
    modeapi.delete()
    modeapi.addrect()
    modeapi.addtogroup(dev_layer)
    modeapi.set("name", f"waveguide {str(i + 1)}")
    modeapi.set("z span", wvg_height)
    modeapi.set("y span", yspan)
    modeapi.set("x span", xspan)
    modeapi.set("x",x1)
    modeapi.set("y",y1)
    modeapi.set("material", material_Si)
    modeapi.set("z", wvg_height / 2)
    
    modeapi.select(f"{dev_layer}::curved taper {str(i + 1)}")
    modeapi.delete()
    modeapi.addcustom()
    modeapi.set("create 3D object by","extrusion")
    modeapi.addtogroup(dev_layer)
    modeapi.set("name", f"curved taper {str(i + 1)}")
    -1.335650270635493*(11.92424001771182-x)^0.5333330000000000+7.499999999999999
    
        

In [94]:
modeapi.addvarfdtd()